In [ ]:
# More data when using the query string below and taking out http.method and http.virtual_host

## s = searchContext.query('query_string', query='event.module:zeek')

##  Still unable to find Empire Beacon 192.168.120.158

In [44]:
#Elasticsearch connector
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import eland as ed
#Data manipulation
import pandas as pd
from pandas.io.json import json_normalize
import json
import re
import eland as ed
import numpy as np
import matplotlib.pyplot as plt
import math
from datetime import datetime
# Disable warnings"
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [45]:
es = Elasticsearch(['https://192.168.1.181:9200'], timeout=30, max_retries=10, retry_on_timeout=True,                  
ca_certs=False,verify_certs=False, http_auth=('jupyter','password'))

# defining the search statement to get all records in an index
searchContext = Search(using=es, index='*:so-*', doc_type='doc')
es.info()

ObjectApiResponse({'name': 'voodoo-onion', 'cluster_name': 'voodoo-onion', 'cluster_uuid': 'dP1VycCSTQ2EPeCVxQ25GA', 'version': {'number': '7.16.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '2b937c44140b6559905130a8650c64dbd0879cfb', 'build_date': '2021-12-18T19:42:46.604893745Z', 'build_snapshot': False, 'lucene_version': '8.10.1', 'minimum_wire_compatibility_version': '6.8.0', 'minimum_index_compatibility_version': '6.0.0-beta1'}, 'tagline': 'You Know, for Search'})

In [46]:
query_since_ts = 0
body_filter = '{"query":{"bool":{"filter":[{"range":{@timestamp":{"gt":' + str(query_since_ts) + '}}}]}},"sort":[{"@timstamp":{"order":"asc"}}]}'
#body_filter = '{"query":{"@timestamp":{}},"size=10000":[{"@timestamp":{"order":"desc"}}]}'

In [47]:
#s = searchContext.query('query_string', query='message:("ts" AND "id.orig_h" AND "id.resp_h" AND "host" AND "id.resp_p" AND "method")')
 
#s = searchContext.query('query_string', query='destination.port:443')

#s = searchContext.query('query_string', query='(destination.port:"*" OR http.virtual_host:"*")')

#s = searchContext.query('query_string', query='http.virtual_host:*')

#s = searchContext.query('query_string', query='http.method:*')

s = searchContext.query('query_string', query='event.module:zeek')


In [48]:
pd.set_option("display.max_rows", 100)
pd.set_option("max_colwidth", 600)
pd.set_option("display.max_columns", None)


#base response query
#response = s.execute()
#if response.success():
#  df = pd.DataFrame((d.to_dict() for d in s.scan()))
#json_struct = json.loads(df.to_json(orient="records"))
#df_flat = pd.io.json.json_normalize(json_struct)
#df_flat.head(20)

#response = s.execute()
#if response.success():
    #df = pd.DataFrame(([d['@timestamp'], d['source']['ip'], d['destination']['ip'], d['http']['virtual_host'], d['destination']['port'], d['http']['method']] for d in s))
#df.columns=['Timestamp','Source.ip','Destination.ip','Host','Destination.port','Method']    
#df.astype({'Destination.Port': 'int64'})
#display(df)
#df.info()

In [49]:
response = s.execute()
df = pd.DataFrame(columns=['ts','source.ip','destination.ip','destination.port'])
if response.success():
    for d in s[:10000]:
        try:
            df = df.append({'ts' : d['@timestamp'], 'source.ip' : d['source']['ip'], 'destination.ip' : d['destination']['ip'], 'destination.port': d['destination']['port']}, ignore_index=True)
        except KeyError:
            pass
            
#response = s.execute()
#if response.success():
#    df = pd.DataFrame(([d['@timestamp'], d['source']['ip'], d['destination']['ip'], d['http']['virtual_host'], d['destination']['port'], d['http']['method']] for d in s))
#df.columns=['Timestamp','Source.ip','Destination.ip','Host','Destination.port','Method']    
#df.astype({'Destination.Port': 'int64'})

display(df)

df.info()

,ts,source.ip,destination.ip,destination.port
0,2021-12-29T22:45:57.681Z,192.168.1.220,224.0.0.252,5355
1,2021-12-29T22:46:00.067Z,192.168.1.220,224.0.0.252,5355
2,2021-12-29T22:46:00.067Z,192.168.1.220,224.0.0.252,5355
3,2021-12-29T22:46:30.090Z,192.168.1.154,13.107.222.240,53
4,2021-12-29T22:46:21.403Z,192.168.1.220,224.0.0.252,5355
...,...,...,...,...
9339,2021-12-29T22:24:47.761Z,192.168.1.220,192.168.1.255,137
9340,2021-12-29T22:23:58.908Z,192.168.1.220,192.168.1.255,137
9341,2021-12-29T22:24:58.078Z,192.168.1.220,192.168.1.255,137
9342,2021-12-29T22:23:57.374Z,192.168.1.220,192.168.1.255,137


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9344 entries, 0 to 9343
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ts                9344 non-null   object
 1   source.ip         9344 non-null   object
 2   destination.ip    9344 non-null   object
 3   destination.port  9344 non-null   object
dtypes: object(4)
memory usage: 292.1+ KB


In [50]:
df['ts'] = pd.to_datetime(df['ts'],infer_datetime_format=True)
df.head(5)

,ts,source.ip,destination.ip,destination.port
0,2021-12-29 22:45:57.681000+00:00,192.168.1.220,224.0.0.252,5355
1,2021-12-29 22:46:00.067000+00:00,192.168.1.220,224.0.0.252,5355
2,2021-12-29 22:46:00.067000+00:00,192.168.1.220,224.0.0.252,5355
3,2021-12-29 22:46:30.090000+00:00,192.168.1.154,13.107.222.240,53
4,2021-12-29 22:46:21.403000+00:00,192.168.1.220,224.0.0.252,5355


In [33]:
# assign field/column names to variables
f_timestamp = 'ts'
f_src_ip = 'source.ip'
f_dst_ip = 'destination.ip'
f_dst_port = 'destination.port'
f_delimiter = '\t'

columns_to_filter = [f_timestamp, f_src_ip, f_dst_ip, f_dst_port,]
columns_to_groupby = [f_src_ip, f_dst_ip, f_dst_port,]
# columns to display after the analysis
columns_to_display = ['tsScore','conn_count',f_src_ip,f_dst_ip,f_dst_port,'deltas']

In [34]:
# If you have a large dateset, using groupby and aggregate(list) might be slow. Consider using dask
df = df.groupby(columns_to_groupby).agg(list)
df.head(10)

ts
source.ip     destination.ip  destination.port                                                                                                                                                                                                                                                
127.0.0.1     127.0.0.1       47761                                                                                                                                                                                                                         [2021-12-29 20:38:42.805000+00:00]
                              55846                                                                                                                                                                                                                         [2021-12-29 20:17:52.147000+00:00]
172.17.0.15   192.168.1.181   9200                                                                                                                                                                                        [2021-12-29 22:05:20.136000+00:00, 2021-12-29 22:05:20.093000+00:00]
192.168.1.154 104.211.91.221  443                                                                                                                                                                                                                           [2021-12-29 20:54:23.101000+00:00]
              104.47.2.8      53                                                                                                                                                        [2021-12-29 21:53:29.882000+00:00, 2021-12-29 21:53:19.294000+00:00, 2021-12-29 20:54:22.626000+00:00]
              104.47.38.8     53                                                                                                                                                                                                                            [2021-12-29 22:16:39.105000+00:00]
              104.47.40.8     53                                                                                                                                                        [2021-12-29 21:53:27.002000+00:00, 2021-12-29 21:52:38.360000+00:00, 2021-12-29 21:55:50.393000+00:00]
              104.80.74.140   80                                                                                                                                                                                                                            [2021-12-29 21:59:27.749000+00:00]
              108.162.193.219 53                                                                                                                                                                                                                            [2021-12-29 21:59:27.523000+00:00]
              13.107.160.4    53                [2021-12-29 22:42:05.804000+00:00, 2021-12-29 21:53:20.584000+00:00, 2021-12-29 22:29:46.086000+00:00, 2021-12-29 20:28:05.027000+00:00, 2021-12-29 20:41:45.045000+00:00, 2021-12-29 21:38:56.859000+00:00, 2021-12-29 21:44:15.358000+00:00]

In [35]:
df.reset_index(inplace=True)
df.head(5)

,source.ip,destination.ip,destination.port,ts
0,127.0.0.1,127.0.0.1,47761,[2021-12-29 20:38:42.805000+00:00]
1,127.0.0.1,127.0.0.1,55846,[2021-12-29 20:17:52.147000+00:00]
2,172.17.0.15,192.168.1.181,9200,"[2021-12-29 22:05:20.136000+00:00, 2021-12-29 22:05:20.093000+00:00]"
3,192.168.1.154,104.211.91.221,443,[2021-12-29 20:54:23.101000+00:00]
4,192.168.1.154,104.47.2.8,53,"[2021-12-29 21:53:29.882000+00:00, 2021-12-29 21:53:19.294000+00:00, 2021-12-29 20:54:22.626000+00:00]"


In [36]:
# create a new column 'conn_count', and for each row in the 'ts' column, apply a function and assign the returned value to the 'conn_count' column
df['conn_count'] = df[f_timestamp].apply(lambda x: len(x))
df.head(2)

,source.ip,destination.ip,destination.port,ts,conn_count
0,127.0.0.1,127.0.0.1,47761,[2021-12-29 20:38:42.805000+00:00],1
1,127.0.0.1,127.0.0.1,55846,[2021-12-29 20:17:52.147000+00:00],1


In [37]:
#Filter out traffic where the connection count is quite small
df = df.loc[df['conn_count'] > 5]
df.shape

(76, 5)

In [38]:
df[f_timestamp] = df[f_timestamp].apply(lambda x: sorted(x))
df.head()

,source.ip,destination.ip,destination.port,ts,conn_count
9,192.168.1.154,13.107.160.4,53,"[2021-12-29 20:28:05.027000+00:00, 2021-12-29 20:41:45.045000+00:00, 2021-12-29 21:38:56.859000+00:00, 2021-12-29 21:44:15.358000+00:00, 2021-12-29 21:53:20.584000+00:00, 2021-12-29 22:29:46.086000+00:00, 2021-12-29 22:42:05.804000+00:00]",7
11,192.168.1.154,13.107.222.240,53,"[2021-12-29 20:00:57.058000+00:00, 2021-12-29 20:02:13.364000+00:00, 2021-12-29 20:05:25.054000+00:00, 2021-12-29 20:05:25.700000+00:00, 2021-12-29 20:21:59.674000+00:00, 2021-12-29 20:25:02.921000+00:00, 2021-12-29 20:28:05.028000+00:00, 2021-12-29 20:37:53.976000+00:00, 2021-12-29 20:37:53.976000+00:00, 2021-12-29 20:38:56.732000+00:00, 2021-12-29 20:41:45.075000+00:00, 2021-12-29 20:42:04.614000+00:00, 2021-12-29 20:43:22.873000+00:00, 2021-12-29 20:43:23.794000+00:00, 2021-12-29 20:45:06.806000+00:00, 2021-12-29 21:00:07.004000+00:00, 2021-12-29 21:02:13.736000+00:00, 2021-12-29 21:12:...",61
31,192.168.1.154,162.208.119.41,443,"[2021-12-29 21:56:54.687000+00:00, 2021-12-29 21:57:17.235000+00:00, 2021-12-29 22:39:42.708000+00:00, 2021-12-29 22:39:42.755000+00:00, 2021-12-29 22:39:59.916000+00:00, 2021-12-29 22:40:46.435000+00:00, 2021-12-29 22:41:48.275000+00:00]",7
37,192.168.1.154,184.85.168.170,443,"[2021-12-29 21:53:23.966000+00:00, 2021-12-29 21:53:23.966000+00:00, 2021-12-29 21:53:24.006000+00:00, 2021-12-29 21:53:24.006000+00:00, 2021-12-29 21:53:24.377000+00:00, 2021-12-29 21:53:24.534000+00:00, 2021-12-29 21:53:24.580000+00:00, 2021-12-29 21:53:24.590000+00:00, 2021-12-29 22:18:40.492000+00:00, 2021-12-29 22:18:40.492000+00:00, 2021-12-29 22:18:40.497000+00:00, 2021-12-29 22:18:40.498000+00:00, 2021-12-29 22:18:40.498000+00:00, 2021-12-29 22:18:40.530000+00:00, 2021-12-29 22:18:40.539000+00:00, 2021-12-29 22:18:40.735000+00:00, 2021-12-29 22:18:40.776000+00:00, 2021-12-29 22:18:...",19
43,192.168.1.154,192.168.1.181,5044,"[2021-12-29 20:00:08.761000+00:00, 2021-12-29 20:00:12.079000+00:00, 2021-12-29 20:00:58.367000+00:00, 2021-12-29 20:00:59.919000+00:00, 2021-12-29 20:01:11.154000+00:00, 2021-12-29 20:01:22.482000+00:00, 2021-12-29 20:01:26.173000+00:00, 2021-12-29 20:02:02.361000+00:00, 2021-12-29 20:02:25.109000+00:00, 2021-12-29 20:02:36.732000+00:00, 2021-12-29 20:03:03.717000+00:00, 2021-12-29 20:03:04.928000+00:00, 2021-12-29 20:03:18.740000+00:00, 2021-12-29 20:03:46.711000+00:00, 2021-12-29 20:04:08.280000+00:00, 2021-12-29 20:04:32.186000+00:00, 2021-12-29 20:04:34.383000+00:00, 2021-12-29 20:04:...",386


In [39]:
# Convert list into a Series object, get time delta, convert the result back into a list and assign it to the 'deltas' column
df['deltas'] = df[f_timestamp].apply(lambda x: pd.Series(x).diff().dt.seconds.dropna().tolist())
df.head(2)

,source.ip,destination.ip,destination.port,ts,conn_count,deltas
9,192.168.1.154,13.107.160.4,53,"[2021-12-29 20:28:05.027000+00:00, 2021-12-29 20:41:45.045000+00:00, 2021-12-29 21:38:56.859000+00:00, 2021-12-29 21:44:15.358000+00:00, 2021-12-29 21:53:20.584000+00:00, 2021-12-29 22:29:46.086000+00:00, 2021-12-29 22:42:05.804000+00:00]",7,"[820.0, 3431.0, 318.0, 545.0, 2185.0, 739.0]"
11,192.168.1.154,13.107.222.240,53,"[2021-12-29 20:00:57.058000+00:00, 2021-12-29 20:02:13.364000+00:00, 2021-12-29 20:05:25.054000+00:00, 2021-12-29 20:05:25.700000+00:00, 2021-12-29 20:21:59.674000+00:00, 2021-12-29 20:25:02.921000+00:00, 2021-12-29 20:28:05.028000+00:00, 2021-12-29 20:37:53.976000+00:00, 2021-12-29 20:37:53.976000+00:00, 2021-12-29 20:38:56.732000+00:00, 2021-12-29 20:41:45.075000+00:00, 2021-12-29 20:42:04.614000+00:00, 2021-12-29 20:43:22.873000+00:00, 2021-12-29 20:43:23.794000+00:00, 2021-12-29 20:45:06.806000+00:00, 2021-12-29 21:00:07.004000+00:00, 2021-12-29 21:02:13.736000+00:00, 2021-12-29 21:12:...",61,"[76.0, 191.0, 0.0, 993.0, 183.0, 182.0, 588.0, 0.0, 62.0, 168.0, 19.0, 78.0, 0.0, 103.0, 900.0, 126.0, 591.0, 594.0, 183.0, 0.0, 833.0, 14.0, 399.0, 332.0, 75.0, 40.0, 2.0, 0.0, 7.0, 0.0, 0.0, 45.0, 3.0, 68.0, 47.0, 425.0, 13.0, 2.0, 0.0, 178.0, 106.0, 145.0, 77.0, 526.0, 1.0, 0.0, 0.0, 67.0, 1.0, 0.0, 144.0, 0.0, 320.0, 21.0, 38.0, 505.0, 110.0, 14.0, 0.0, 48.0]"


In [40]:
df['tsLow'] = df['deltas'].apply(lambda x: np.percentile(np.array(x), 20))
df['tsMid'] = df['deltas'].apply(lambda x: np.percentile(np.array(x), 50))
df['tsHigh'] = df['deltas'].apply(lambda x: np.percentile(np.array(x), 80))
df['tsBowleyNum'] = df['tsLow'] + df['tsHigh'] - 2*df['tsMid']
df['tsBowleyDen'] = df['tsHigh'] - df['tsLow']
df['tsSkew'] = df[['tsLow','tsMid','tsHigh','tsBowleyNum','tsBowleyDen']].apply(
    lambda x: x['tsBowleyNum'] / x['tsBowleyDen'] if x['tsBowleyDen'] != 0 and x['tsMid'] != x['tsLow'] and x['tsMid'] != x['tsHigh'] else 0.0, axis=1
    )
df['tsMadm'] = df['deltas'].apply(lambda x: np.median(np.absolute(np.array(x) - np.median(np.array(x)))))
df['tsConnDiv'] = df[f_timestamp].apply(lambda x: (x[-1].to_pydatetime() - x[0].to_pydatetime()).seconds / 10)

In [41]:
df.head(3)

,source.ip,destination.ip,destination.port,ts,conn_count,deltas,tsLow,tsMid,tsHigh,tsBowleyNum,tsBowleyDen,tsSkew,tsMadm,tsConnDiv
9,192.168.1.154,13.107.160.4,53,"[2021-12-29 20:28:05.027000+00:00, 2021-12-29 20:41:45.045000+00:00, 2021-12-29 21:38:56.859000+00:00, 2021-12-29 21:44:15.358000+00:00, 2021-12-29 21:53:20.584000+00:00, 2021-12-29 22:29:46.086000+00:00, 2021-12-29 22:42:05.804000+00:00]",7,"[820.0, 3431.0, 318.0, 545.0, 2185.0, 739.0]",545.0,779.5,2185.0,1171.0,1640.0,0.714024,348.0,804.0
11,192.168.1.154,13.107.222.240,53,"[2021-12-29 20:00:57.058000+00:00, 2021-12-29 20:02:13.364000+00:00, 2021-12-29 20:05:25.054000+00:00, 2021-12-29 20:05:25.700000+00:00, 2021-12-29 20:21:59.674000+00:00, 2021-12-29 20:25:02.921000+00:00, 2021-12-29 20:28:05.028000+00:00, 2021-12-29 20:37:53.976000+00:00, 2021-12-29 20:37:53.976000+00:00, 2021-12-29 20:38:56.732000+00:00, 2021-12-29 20:41:45.075000+00:00, 2021-12-29 20:42:04.614000+00:00, 2021-12-29 20:43:22.873000+00:00, 2021-12-29 20:43:23.794000+00:00, 2021-12-29 20:45:06.806000+00:00, 2021-12-29 21:00:07.004000+00:00, 2021-12-29 21:02:13.736000+00:00, 2021-12-29 21:12:...",61,"[76.0, 191.0, 0.0, 993.0, 183.0, 182.0, 588.0, 0.0, 62.0, 168.0, 19.0, 78.0, 0.0, 103.0, 900.0, 126.0, 591.0, 594.0, 183.0, 0.0, 833.0, 14.0, 399.0, 332.0, 75.0, 40.0, 2.0, 0.0, 7.0, 0.0, 0.0, 45.0, 3.0, 68.0, 47.0, 425.0, 13.0, 2.0, 0.0, 178.0, 106.0, 145.0, 77.0, 526.0, 1.0, 0.0, 0.0, 67.0, 1.0, 0.0, 144.0, 0.0, 320.0, 21.0, 38.0, 505.0, 110.0, 14.0, 0.0, 48.0]",0.0,64.5,216.8,87.8,216.8,0.404982,64.5,966.8
31,192.168.1.154,162.208.119.41,443,"[2021-12-29 21:56:54.687000+00:00, 2021-12-29 21:57:17.235000+00:00, 2021-12-29 22:39:42.708000+00:00, 2021-12-29 22:39:42.755000+00:00, 2021-12-29 22:39:59.916000+00:00, 2021-12-29 22:40:46.435000+00:00, 2021-12-29 22:41:48.275000+00:00]",7,"[22.0, 2545.0, 0.0, 17.0, 46.0, 61.0]",17.0,34.0,61.0,10.0,44.0,0.227273,22.0,269.3


In [42]:
df['tsSkewScore'] = 1.0 - abs(df['tsSkew'])
df['tsMadmScore'] = 1.0 - df['tsMadm']/30.0
df['tsMadmScore'] = df['tsMadmScore'].apply(lambda x: 0 if x < 0 else x)
df['tsConnCountScore'] = (df['conn_count']) / df['tsConnDiv']
df['tsConnCountScore'] = df['tsConnCountScore'].apply(lambda x: 1.0 if x > 1.0 else x)
df['tsScore'] = (((df['tsSkewScore'] + df['tsMadmScore'] + df['tsConnCountScore']) / 3.0) * 1000) / 1000
df.sort_values(by= 'tsScore', ascending=False, inplace=True, ignore_index=True)
df[columns_to_display].head(30)

,tsScore,conn_count,source.ip,destination.ip,destination.port,deltas
0,1.000000,9,192.168.128.30,224.0.0.252,5355,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0]"
1,1.000000,33,192.168.1.154,8.240.196.126,80,"[2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
2,1.000000,8,fe80::29a2:1b55:ba41:8aa1,ff02::1:3,5355,"[0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0]"
3,1.000000,37,192.168.128.50,8.240.196.126,80,"[0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
4,1.000000,72,192.168.128.15,23.66.101.177,80,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
5,1.000000,6,192.168.1.154,8.240.225.126,80,"[0.0, 0.0, 0.0, 0.0, 0.0]"
6,0.877778,3061,192.168.1.220,192.168.1.255,137,"[0.0, 6.0, 1.0, 0.0, 2.0, 1.0, 0.0, 1.0, 0.0, 0.0, 3.0, 1.0, 7.0, 2.0, 1.0, 0.0, 0.0, 4.0, 2.0, 7.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 8.0, 8.0, 0.0, 32.0, 7.0, 0.0, 1.0, 0.0, 1.0, 0.0, 2.0, 3.0, 3.0, 0.0, 7.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 2.0, 1.0, 0.0, 0.0, 0.0, 1.0, 6.0, 0.0, 0.0, 1.0, 6.0, 2.0, 0.0, 8.0, 0.0, 2.0, 0.0, 0.0, 29.0, 15.0, 0.0, 0.0, 0.0, 2.0, 1.0, 0.0, 6.0, 6.0, 0.0, 0.0, 0.0, 0.0, 1.0, 3.0, 0.0, 6.0, 2.0, 1.0, 4.0, 1.0, 3.0, 2.0, 6.0, 0.0, 1.0, 0.0, 31.0, 7.0, 0.0, 0.0, 0.0, 2.0, ...]"
7,0.834921,1619,192.168.1.220,224.0.0.252,5355,"[2.0, 11.0, 0.0, 5.0, 9.0, 9.0, 0.0, 2.0, 4.0, 0.0, 14.0, 0.0, 31.0, 6.0, 0.0, 1.0, 2.0, 5.0, 1.0, 0.0, 2.0, 6.0, 2.0, 0.0, 4.0, 0.0, 2.0, 2.0, 1.0, 0.0, 9.0, 4.0, 5.0, 1.0, 9.0, 32.0, 6.0, 5.0, 4.0, 0.0, 2.0, 2.0, 6.0, 4.0, 4.0, 11.0, 10.0, 2.0, 2.0, 6.0, 44.0, 11.0, 11.0, 28.0, 2.0, 0.0, 7.0, 1.0, 32.0, 0.0, 0.0, 8.0, 5.0, 1.0, 5.0, 9.0, 0.0, 0.0, 1.0, 0.0, 2.0, 2.0, 0.0, 2.0, 33.0, 32.0, 7.0, 9.0, 1.0, 2.0, 2.0, 0.0, 0.0, 3.0, 2.0, 0.0, 3.0, 2.0, 2.0, 11.0, 0.0, 4.0, 4.0, 55.0, 0.0, 4.0, 0.0, 0.0, 4.0, 14.0, ...]"
8,0.833333,1035,192.168.1.220,224.0.0.251,5353,"[7.0, 1.0, 3.0, 2.0, 4.0, 9.0, 4.0, 1.0, 5.0, 10.0, 0.0, 2.0, 2.0, 1.0, 11.0, 1.0, 2.0, 53.0, 13.0, 8.0, 6.0, 1.0, 1.0, 3.0, 2.0, 2.0, 2.0, 1.0, 26.0, 14.0, 7.0, 1.0, 20.0, 1.0, 3.0, 8.0, 91.0, 1.0, 1.0, 1.0, 22.0, 17.0, 1.0, 32.0, 9.0, 3.0, 3.0, 4.0, 9.0, 12.0, 17.0, 55.0, 2.0, 1.0, 9.0, 0.0, 7.0, 0.0, 2.0, 2.0, 12.0, 1.0, 4.0, 1.0, 1.0, 5.0, 8.0, 45.0, 4.0, 4.0, 5.0, 8.0, 1.0, 3.0, 13.0, 3.0, 49.0, 27.0, 2.0, 31.0, 8.0, 284.0, 34.0, 1.0, 19.0, 1.0, 5.0, 9.0, 1.0, 9.0, 0.0, 2.0, 17.0, 2.0, 5.0, 23.0, 34.0, 8.0, 7.0, 15.0, ...]"
9,0.816479,8,192.168.128.30,192.168.128.25,389,"[1.0, 176.0, 0.0, 0.0, 0.0, 0.0, 0.0]"


In [43]:
df.loc[df['tsScore'] > 0.80, columns_to_display]

,tsScore,conn_count,source.ip,destination.ip,destination.port,deltas
0,1.000000,9,192.168.128.30,224.0.0.252,5355,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0]"
1,1.000000,33,192.168.1.154,8.240.196.126,80,"[2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
2,1.000000,8,fe80::29a2:1b55:ba41:8aa1,ff02::1:3,5355,"[0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0]"
3,1.000000,37,192.168.128.50,8.240.196.126,80,"[0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
4,1.000000,72,192.168.128.15,23.66.101.177,80,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
5,1.000000,6,192.168.1.154,8.240.225.126,80,"[0.0, 0.0, 0.0, 0.0, 0.0]"
6,0.877778,3061,192.168.1.220,192.168.1.255,137,"[0.0, 6.0, 1.0, 0.0, 2.0, 1.0, 0.0, 1.0, 0.0, 0.0, 3.0, 1.0, 7.0, 2.0, 1.0, 0.0, 0.0, 4.0, 2.0, 7.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 8.0, 8.0, 0.0, 32.0, 7.0, 0.0, 1.0, 0.0, 1.0, 0.0, 2.0, 3.0, 3.0, 0.0, 7.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 2.0, 1.0, 0.0, 0.0, 0.0, 1.0, 6.0, 0.0, 0.0, 1.0, 6.0, 2.0, 0.0, 8.0, 0.0, 2.0, 0.0, 0.0, 29.0, 15.0, 0.0, 0.0, 0.0, 2.0, 1.0, 0.0, 6.0, 6.0, 0.0, 0.0, 0.0, 0.0, 1.0, 3.0, 0.0, 6.0, 2.0, 1.0, 4.0, 1.0, 3.0, 2.0, 6.0, 0.0, 1.0, 0.0, 31.0, 7.0, 0.0, 0.0, 0.0, 2.0, ...]"
7,0.834921,1619,192.168.1.220,224.0.0.252,5355,"[2.0, 11.0, 0.0, 5.0, 9.0, 9.0, 0.0, 2.0, 4.0, 0.0, 14.0, 0.0, 31.0, 6.0, 0.0, 1.0, 2.0, 5.0, 1.0, 0.0, 2.0, 6.0, 2.0, 0.0, 4.0, 0.0, 2.0, 2.0, 1.0, 0.0, 9.0, 4.0, 5.0, 1.0, 9.0, 32.0, 6.0, 5.0, 4.0, 0.0, 2.0, 2.0, 6.0, 4.0, 4.0, 11.0, 10.0, 2.0, 2.0, 6.0, 44.0, 11.0, 11.0, 28.0, 2.0, 0.0, 7.0, 1.0, 32.0, 0.0, 0.0, 8.0, 5.0, 1.0, 5.0, 9.0, 0.0, 0.0, 1.0, 0.0, 2.0, 2.0, 0.0, 2.0, 33.0, 32.0, 7.0, 9.0, 1.0, 2.0, 2.0, 0.0, 0.0, 3.0, 2.0, 0.0, 3.0, 2.0, 2.0, 11.0, 0.0, 4.0, 4.0, 55.0, 0.0, 4.0, 0.0, 0.0, 4.0, 14.0, ...]"
8,0.833333,1035,192.168.1.220,224.0.0.251,5353,"[7.0, 1.0, 3.0, 2.0, 4.0, 9.0, 4.0, 1.0, 5.0, 10.0, 0.0, 2.0, 2.0, 1.0, 11.0, 1.0, 2.0, 53.0, 13.0, 8.0, 6.0, 1.0, 1.0, 3.0, 2.0, 2.0, 2.0, 1.0, 26.0, 14.0, 7.0, 1.0, 20.0, 1.0, 3.0, 8.0, 91.0, 1.0, 1.0, 1.0, 22.0, 17.0, 1.0, 32.0, 9.0, 3.0, 3.0, 4.0, 9.0, 12.0, 17.0, 55.0, 2.0, 1.0, 9.0, 0.0, 7.0, 0.0, 2.0, 2.0, 12.0, 1.0, 4.0, 1.0, 1.0, 5.0, 8.0, 45.0, 4.0, 4.0, 5.0, 8.0, 1.0, 3.0, 13.0, 3.0, 49.0, 27.0, 2.0, 31.0, 8.0, 284.0, 34.0, 1.0, 19.0, 1.0, 5.0, 9.0, 1.0, 9.0, 0.0, 2.0, 17.0, 2.0, 5.0, 23.0, 34.0, 8.0, 7.0, 15.0, ...]"
9,0.816479,8,192.168.128.30,192.168.128.25,389,"[1.0, 176.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
